In [2]:
%pip -q install google-genai

In [1]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [2]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura ainda não anunciou uma nova edição da Imersão IA com Google Gemini. A última edição ocorreu em fevereiro de 2024.

A melhor forma de ficar sabendo sobre futuras edições é:

*   **Seguir a Alura nas redes sociais:** Eles costumam anunciar novidades por lá (LinkedIn, Instagram, etc.).
*   **Assinar a newsletter da Alura:** Assim você recebe informações diretamente no seu e-mail.
*   **Acompanhar o fórum da Alura:** A comunidade pode compartilhar informações sobre novos cursos e imersões.
*   **Verificar a página de cursos de IA da Alura:** Fique de olho na página de cursos de Inteligência Artificial da Alura para ver se há alguma novidade ou previsão de novas turmas.

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Insere a tool de busca do Google ###############################################
    config={"tools":[{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para o curso já estão abertas no site da Alura. O curso é gratuito, 100% online e voltado para profissionais de todas as áreas, sem exigir conhecimento prévio. Ao final da imersão, os participantes ganham um certificado da Alura e do Google.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, alura.com.br, youtube.com



In [7]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [14]:
##################################
# --- Agente 1: Brainstormer --- #
##################################
def agente_brainstormer(briefing):
    brainstormer = Agent(
        name="agente_brainstormer",
        model="gemini-2.0-flash",
        # Instruções do Agente de Brainstorming #################################################
        instruction="""
          Haja como um time de brainstorming composto por diferentes perfis (O Pensador Criativo,
          O Analista, O Executor, O Entusiasta, O Cético Construtivo, O Especialista). Você deve levantar ideias
          de ações com base no briefing fornecido. O evento pode ser desde uma festa onde cada funcionário
          pode trazer um item até uma festa épica em uma supercasa de eventos. Pode ter decorações temáticas,
          convites especiais, impressos, banners, espaços instagramáveis, materiais de divulgação etc.,
          desde que se adeque ao orçamento e requisitos fornecidos.  Você pode utilizar a ferramenta do Google (google_search)
          para buscar informações sobre o tema pesquisar ideias, locais, fornecedores, brindes, materiais etc.
          Ao final elenque as 5 melhores e mais criativas ideias de festa que podem servir ao tema proposto no briefing. Inclua o
          slogan, as possíveis ações e atrações, ideias de materiais, brindes, peças de comunicação, decorações, alimentação etc.).
        """,
      description="Agente de brainstorming que gera ideias iniciais de eventos.",
      tools=[google_search]
    )
    entrada_do_agente_brainstorming = f"Briefing:{briefing}"
    lista_de_ideias = call_agent(brainstormer, entrada_do_agente_brainstorming)
    return lista_de_ideias

In [44]:
###########################################
# --- Agente 2: Planejador de eventos --- #
###########################################
def agente_planejador(briefing, lista_de_ideias, data_de_hoje):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
          Você é um especialista em eventos dentro do time de comunicação interna e endomarketing da PaperCare.
          Seu objetivo é analisar as ideias iniciais levantadas na etapa de brainstorming e organizar o evento: criar a programação,
          levantar as peças de comunicação necessárias, levantar custos e elaborar o cronograma de execução (compra de itens,
          produção das peças de comunicação, minuto a minuto do evento etc.). Você pode mesclar itens de duas ou mais ideias no
          planejamento do evento. Lembre-se de considerar a data de hoje e a data do evento no planejamento. Utilize a pesquisa do
          Google (google_search) para buscar informações complementares para refinar o planejamento. Você pode contar com os canais
          de comunicação da empresa (e-mail, murais de avisos, fundo de telas, intranet e TVs) e com o time de comunicação que conta
          com pessoas que podem apoiar com desenvolvimento das artes. Ao final, gere uma apresentação com a proposta do evento. Essa apresentação
          deverá conter um resumo sobre o evento, trazer o tema e slogan, programação do evento, plano de comunicação, plano de execução
          e custos(importante especificar os nomes dos itens, quantidades, valores unitários e valores totais). Essa apresentação servirá para aprovação do evento pela gestão da empresa e será utilizada como base para criação dos
          textos e peças de comunicação pelo time de comunicação.
        """,
        description="Agente que planeja os eventos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Briefing:{briefing}\nLista de Ideias: {lista_de_ideias}\nData de início do Planejamento:{data_de_hoje}"
    # Executa o agente
    plano_do_evento = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_evento

In [37]:
#################################
# --- Agente 3: Comunicação --- #
#################################
def agente_comunicador(briefing, plano_do_evento):
    comunicador = Agent(
        name="agente_comunicador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um especialista de comunicação da PaperCare. Sua função é elaborar os textos
            para as peças de comunicação propostas no plano do evento. As comunicações devem ser integradas
            entre si e com o core da empresa. Os textos devem levar em consideração as especificações
            fornecidas pelo briefing e plano no evento.
            Os textos devem ser de leitura rápida, engajadores, informativo, com linguagem simples e incluir 3 a 5 hashtags no final.
            Todos os materiais, preferencialmente, devem ter o mesmo título e slogan para reforçar o propósito do evento e comunicação.
            """,
        description="Agente criador dos textos das comunicações do evento."
    )
    entrada_do_agente_comunicador = f"Briefing:{briefing}\nPlano do Evento: {plano_do_evento}"
    # Executa o agente
    draft_comunicacao = call_agent(comunicador, entrada_do_agente_comunicador)
    return draft_comunicacao

In [38]:
#######################################
# --- Agente 4: Revisor do Evento --- #
#######################################
def agente_revisor(briefing, plano_do_evento,draft_comunicacao):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é o gestor e especialista em comunicação, endomarketing e experiencia do colaborador
            da PaperCare. Sua função é revisar os drafts das comunicações e verificar se as comunicações
            estão alinhadas com o Briefing e Plano do Evento. Caso haja melhorias, sugira alterações.
            Caso esteja tudo certo, sinalize apenas com “Aprovado! Podem seguir.”
            """,
        description="Agente revisor da comunicação do evento."
    )
    entrada_do_agente_revisor = f"Briefing:{briefing}\nPlano do Evento: {plano_do_evento}\nDraft da Comunicação: {draft_comunicacao}"
    # Executa o agente
    comunicacao_revisada = call_agent(revisor, entrada_do_agente_revisor)
    return comunicacao_revisada

In [42]:
#######################################
# --- Formulário de Briefing --- #
#######################################

def coletar_briefing():
    #Coleta informações do briefing do evento corporativo através de inputs validados.

    briefing = {}

    # Tipo de evento
    while True:
        tipo_evento = input("Qual o tipo de evento (Festa de final de ano, convenção, aniversário da empresa, campanha etc.)? ").strip()
        if tipo_evento:
            briefing['tipo_evento'] = tipo_evento
            break
        else:
            print("Por favor, informe o tipo de evento.")

    # Tema
    briefing['tema'] = input("Qual é o tema? ").strip()

    # Data/período do evento
    while True:
        data_periodo = input("Data/período do evento? ").strip()
        if data_periodo:
            briefing['data_periodo'] = data_periodo
            break
        else:
            print("Por favor, informe a data ou período do evento.")

    # Horário
    while True:
        horario = input("Horário? ").strip()
        if horario:
            briefing['horario'] = horario
            break
        else:
            print("Por favor, informe o horário do evento.")

    # Público do evento - Número de pessoas
    while True:
        try:
            num_pessoas = int(input("Público do evento: Número de pessoas? "))
            if num_pessoas > 0:
                briefing['num_pessoas'] = num_pessoas
                break
            else:
                print("Por favor, informe um número de pessoas válido.")
        except ValueError:
            print("Por favor, insira um número inteiro para a quantidade de pessoas.")

    # Público do evento - Perfil/público-alvo
    briefing['perfil_publico'] = input("Público do evento: Perfil/público-alvo (%mulheres x homens, faixa de idade, alta gestão, toda companhia, pessoal do escritório etc.)? ").strip()

    # Orçamento previsto
    while True:
        try:
            orcamento = float(input("Orçamento previsto em Reais (R$)? "))
            if orcamento >= 0:
                briefing['orcamento'] = orcamento
                break
            else:
                print("Por favor, informe um orçamento válido em reais (R$).")
        except ValueError:
            print("Por favor, insira um valor numérico para o orçamento.")

    # O que precisa ter
    briefing['precisa_ter'] = input("O que precisa ter? ").strip()

    # Restrições
    briefing['restricoes'] = input("Restrições: ").strip()

    # Informações complementares
    info_complementares = """
    · A PaperCare é uma empresa fabricante de papel (consumo, profissional e industrial) e produtos de cuidados pessoais (absorventes e fraldas).
    · Tem cerca de 1200 funcionários distribuídos em 2 plantas fabris (com 870 e 215 funcionários), 1 escritório (85 funcionários) e time comercial (30 funcionários). A empresa tem 57 anos de existência e tem como lema “Nosso papel é construir o futuro”.
    · Perfil do publico: 72% masculino; 28% feminino; 5% LGBTQIAP+; 80% do time das fábricas é operacional.
    · As áreas fabris funcionam 24 horas por dia. Os operadores estão divididos em 4 equipes que rodam turnos de 8 horas. A cada 5 dias essas equipes fazem um rodízio de turno, sendo que uma das equipes fica de folga.
    · O escritório, áreas administrativas e time comercial trabalham em horário comercial. Os times comerciais atuam fora do escritório.
    · Considere que o time de Comunicação e Endomarleting é reduzido para produzir e executar o evento. Composição: 3 integrantes no escritório central + 1 apoio em cada fabrica.
    """

    # Formatar a saída como um texto, incluindo as informações complementares
    texto_briefing = f"""
    Briefing do Evento:

    Tipo de Evento: {briefing['tipo_evento']}
    Tema: {briefing['tema']}
    Data/Período: {briefing['data_periodo']}
    Horário: {briefing['horario']}
    Público - Número de Pessoas: {briefing['num_pessoas']}
    Público - Perfil: {briefing['perfil_publico']}
    Orçamento Previsto(R$): {briefing['orcamento']}
    Precisa Ter: {briefing['precisa_ter']}
    Restrições: {briefing['restricoes']}

    Informações complementares:{info_complementares}
    """

    return texto_briefing

In [45]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("ASSISTENTE BUSINESS PARTY")
print("Olá sou seu assistente personalizado da PaperCare e estou aqui para auxiliá-lo a produzir os eventos da companhia.")

# Inserir lógica do sistema de agentes ################################################

# --- Obter o Briefing do Usuário ---
print("Vamos a primeira etapa. O Briefing:")
briefing = coletar_briefing()

# --- Resumo do Briefing ---
print("\n--- Resumo do Briefing ---\n")
display(to_markdown(briefing))
print("----------------------------------------------------------------")

# --- Etapa de Brainstorming ---
lista_de_ideias = agente_brainstormer(briefing)
print("\n--- Resultado da Etapa de Brainstorming ---\n")
display(to_markdown(lista_de_ideias))
print("----------------------------------------------------------------")

# --- Etapa de Planejamento ---
plano_do_evento = agente_planejador(briefing, lista_de_ideias, data_de_hoje)
print("\n--- Resultado da Etapa de Planejamento ---\n")
display(to_markdown(plano_do_evento))
print("----------------------------------------------------------------")

# --- Etapa de Criação da Comunicação ---
draft_comunicacao = agente_comunicador(briefing, plano_do_evento)
print("\n--- Resultado da Etapa de Criação da Comunicação ---\n")
display(to_markdown(draft_comunicacao))
print("----------------------------------------------------------------")

# --- Etapa de Revisão da Comunicação ---
comunicacao_revisada = agente_revisor(briefing, plano_do_evento,draft_comunicacao)
print("\n--- Resultado da Etapa de Revisão da Comunicação ---\n")
display(to_markdown(comunicacao_revisada))
print("----------------------------------------------------------------")
print(f"ASSISTENTE BUSINESS PARTY - IROTSUKA - {data_de_hoje}")
print(f"####################################################")
print(f"#  LINKEDIN:https://www.linkedin.com/in/irotsuka/  #")
print(f"#  GITHUB: https://github.com/i-otsuka             #")
print(f"####################################################")

ASSISTENTE BUSINESS PARTY
Olá sou seu assistente personalizado da PaperCare e estou aqui para auxiliá-lo a produzir os eventos da companhia.
Vamos a primeira etapa. O Briefing:
Qual o tipo de evento (Festa de final de ano, convenção, aniversário da empresa, campanha etc.)? Convenção de Vendas
Qual é o tema? 2025 o ano da superação
Data/período do evento? 8 a 11 de julhp de 2025.
Horário? Dia 8 das 10h as 17h; 9 e 10 das 9h as 20h; 11 das 9h as 14h.
Público do evento: Número de pessoas? Time comercial mais 30 pessoas das áreas de marketing, recursos humanos e TI.
Por favor, insira um número inteiro para a quantidade de pessoas.
Público do evento: Número de pessoas? 70
Público do evento: Perfil/público-alvo (%mulheres x homens, faixa de idade, alta gestão, toda companhia, pessoal do escritório etc.)? Todo o time comercial, mais integrandes das áreas de marketing, recursos humanos e ti.
Orçamento previsto em Reais (R$)? 40000
O que precisa ter? Reuniões de venda com time de mkt, momento a

> 
>     Briefing do Evento:
> 
>     Tipo de Evento: Convenção de Vendas
>     Tema: 2025 o ano da superação
>     Data/Período: 8 a 11 de julhp de 2025.
>     Horário: Dia 8 das 10h as 17h; 9 e 10 das 9h as 20h; 11 das 9h as 14h.
>     Público - Número de Pessoas: 70
>     Público - Perfil: Todo o time comercial, mais integrandes das áreas de marketing, recursos humanos e ti.
>     Orçamento Previsto(R$): 40000.0
>     Precisa Ter: Reuniões de venda com time de mkt, momento ara troca dos notebooks, atualização cadastral junto com RH.
>     Restrições: Necessário local com acessibilidade pois temos dois funcionários cadeirantes.
> 
>     Informações complementares:
>     · A PaperCare é uma empresa fabricante de papel (consumo, profissional e industrial) e produtos de cuidados pessoais (absorventes e fraldas). 
>     · Tem cerca de 1200 funcionários distribuídos em 2 plantas fabris (com 870 e 215 funcionários), 1 escritório (85 funcionários) e time comercial (30 funcionários). A empresa tem 57 anos de existência e tem como lema “Nosso papel é construir o futuro”.
>     · Perfil do publico: 72% masculino; 28% feminino; 5% LGBTQIAP+; 80% do time das fábricas é operacional.
>     · As áreas fabris funcionam 24 horas por dia. Os operadores estão divididos em 4 equipes que rodam turnos de 8 horas. A cada 5 dias essas equipes fazem um rodízio de turno, sendo que uma das equipes fica de folga.
>     · O escritório, áreas administrativas e time comercial trabalham em horário comercial. Os times comerciais atuam fora do escritório.
>     · Considere que o time de Comunicação e Endomarleting é reduzido para produzir e executar o evento. Composição: 3 integrantes no escritório central + 1 apoio em cada fabrica.
>     
>     

----------------------------------------------------------------

--- Resultado da Etapa de Brainstorming ---



> Okay, vamos gerar algumas ideias criativas para a convenção de vendas da PaperCare com o tema "2025: O Ano da Superação", considerando o orçamento, público e restrições.
> 
> 
> Com base no briefing e nas pesquisas, aqui estão 5 ideias criativas para a convenção de vendas da PaperCare:
> 
> **1. Tema:** PaperCare: Rumo ao Topo! 2025, Nossa Escalada da Superação
> *   **Slogan:** "Juntos, no nosso papel, construímos o futuro no topo!"
> *   **Ações e Atrações:**
>     *   **Decoração:** Cenário que remete a uma escalada, com montanhas (podem ser de papelão ou tecido), cordas, picos nevados (algodão ou isopor), e pegadas no chão. Cores vibrantes e inspiradoras. Espaço instagramável com asas de anjo e a hashtag #PaperCareNoTopo.
>     *   **Atividades:** Dinâmicas de grupo com foco em superação de obstáculos, planejamento estratégico com visualização de metas no topo da montanha, palestra motivacional com um escalador famoso ou especialista em resiliência. Apresentação teatral com a história da PaperCare como uma jornada de superação.
>     *   **Alimentação:** Cardápio que remete a "comida de montanha", com opções energéticas e reconfortantes.
>     *   **Brindes:** Kit de "escalada" com garrafa de água personalizada, barra de cereais, protetor solar, boné e uma mini bússola.
>     *   **Comunicação:** Convites personalizados com um "mapa da mina" da convenção, banners com fotos dos colaboradores "escalando" desafios, vídeos motivacionais com depoimentos dos funcionários.
> 
> **2. Tema:** PaperCare Supera: Nossa Liga de Heróis de Vendas
> *   **Slogan:** "Em 2025, somos a força invencível da PaperCare!"
> *   **Ações e Atrações:**
>     *   **Decoração:** Ambientação inspirada em histórias em quadrinhos de super-heróis. Cores vibrantes, balões de fala com mensagens motivacionais, painel com os "superpoderes" de cada área da empresa (ex: "Visão de Raio-X do Marketing", "Força Inabalável do RH").
>     *   **Atividades:** Cada participante recebe uma "capa" personalizada com seu nome e "superpoder". Apresentação dos resultados do ano como se fossem "missões cumpridas". Dinâmicas de grupo para desenvolver "superpoderes" como comunicação, negociação e trabalho em equipe. Premiação dos "Heróis do Ano" com troféus temáticos.
>     *   **Alimentação:** "Super lanches" com nomes criativos, como "Sanduíche do Hulk", "Energia do Flash", "Suco da Mulher Maravilha".
>     *   **Brindes:** Caneca personalizada com o logo da "Liga PaperCare", power bank com o tema de super-herói, capa de chuva personalizada (afinal, heróis também se protegem da chuva!).
>     *   **Comunicação:** Convites em formato de HQ, teasers com mensagens enigmáticas dos "super-heróis" da PaperCare, backdrop para fotos com a logo da "Liga".
> 
> **3. Tema:** PaperCare 2025: Game On! A Virada do Século
> *   **Slogan:** "Em 2025, PaperCare: Jogamos juntos, vencemos juntos!"
> *   **Ações e Atrações:**
>     *   **Decoração:** Cenário que remete a jogos de videogame, com cores neon, elementos pixelados, e referências a jogos clássicos e atuais. Telões exibindo gráficos e animações.
>     *   **Atividades:** Competição de jogos (virtuais ou de tabuleiro) com temas relacionados ao trabalho, quizzes sobre a história da PaperCare, "hackathons" para encontrar soluções inovadoras para os desafios da empresa. Premiação dos melhores "gamers" com brindes tecnológicos.
>     *   **Alimentação:** Cardápio com "comida de gamer", como pizza, hambúrguer, batata frita e energéticos.
>     *   **Brindes:** Fone de ouvido gamer, mini console de videogame, vale-presente para loja de jogos online, camiseta personalizada com a logo da PaperCare em estilo "gamer".
>     *   **Comunicação:** Convites em formato de "cartão de memória", vídeos com depoimentos dos funcionários em estilo "gameplay", posts nas redes sociais com a hashtag #PaperCareGameOn.
> 
> **4. Tema:** PaperCare Conecta: Nossa Sinfonia de Talentos Rumo a 2025
> *   **Slogan:** "Em 2025, PaperCare: Uma orquestra de talentos, uma sinfonia de sucesso!"
> *   **Ações e Atrações:**
>     *   **Decoração:** Ambientação inspirada em música, com instrumentos musicais, partituras, luzes que remetem a um show. Cores suaves e elegantes.
>     *   **Atividades:** Formação de uma "banda PaperCare" com os funcionários, apresentação de um coral com músicas motivacionais, dinâmicas de grupo para afinar a comunicação e o trabalho em equipe, palestra com um maestro ou músico inspirador.
>     *   **Alimentação:** Cardápio sofisticado com nomes de pratos inspirados em músicas clássicas, como "Allegro de Frutos do Mar", "Sonata de Legumes", "Concerto de Chocolate".
>     *   **Brindes:** Fone de ouvido com cancelamento de ruído, caixinha de música personalizada com o tema da PaperCare, instrumento musical (ex: apito, ukulele) com a logo da empresa.
>     *   **Comunicação:** Convites personalizados com uma "partitura" da convenção, vídeos com depoimentos dos funcionários em estilo "videoclipe", posts nas redes sociais com a hashtag #PaperCareConecta.
> 
> **5. Tema:** PaperCare Sustentável: Plantando o Futuro, Colhendo Superação em 2025
> *   **Slogan:** "Em 2025, PaperCare: Cuidamos do planeta, cultivamos o sucesso!"
> *   **Ações e Atrações:**
>     *   **Decoração:** Ambientação inspirada na natureza, com plantas, flores, árvores (de verdade ou artificiais), materiais reciclados, cores verde e marrom.
>     *   **Atividades:** Oficina de jardinagem, palestra sobre sustentabilidade e responsabilidade social, dinâmicas de grupo para encontrar soluções para reduzir o impacto ambiental da empresa, ação de plantio de árvores em uma área degradada.
>     *   **Alimentação:** Cardápio orgânico e vegetariano, com ingredientes frescos e da estação.
>     *   **Brindes:** Kit de jardinagem com sementes e ferramentas, ecobag personalizada, caneca reutilizável de bambu, squeeze de alumínio.
>     *   **Comunicação:** Convites em papel reciclado com sementes, vídeos com depoimentos dos funcionários sobre suas práticas sustentáveis, posts nas redes sociais com a hashtag #PaperCareSustentável.
> 
> **Considerações Importantes:**
> 
> *   **Acessibilidade:** Todos os locais e atividades devem ser acessíveis para os funcionários cadeirantes, com rampas, elevadores, banheiros adaptados e espaços amplos para circulação.
> *   **Orçamento:** É fundamental pesquisar preços e negociar com fornecedores para garantir que as ideias se encaixem no orçamento de R$40.000. Priorizar o que é mais importante para o evento (ex: local, palestrante, brindes) e buscar alternativas mais econômicas para o restante.
> *   **Público:** Considerar o perfil do público (72% masculino, 28% feminino, 5% LGBTQIAP+, 80% do time das fábricas é operacional) ao escolher as atividades, brindes e peças de comunicação. Buscar temas e atrações que agradem a todos os públicos.
> *   **Restrições de Tempo:** Considerar os horários de trabalho das áreas fabris ao planejar as atividades. Buscar horários e atividades que permitam a participação de todos os funcionários.
> 
> Espero que essas ideias sirvam de inspiração para a convenção de vendas da PaperCare!


----------------------------------------------------------------

--- Resultado da Etapa de Planejamento ---



...

----------------------------------------------------------------

--- Resultado da Etapa de Criação da Comunicação ---



> ## Sugestões de Textos para as Peças de Comunicação da Convenção de Vendas PaperCare 2025
> 
> **Título Comum:** PaperCare: Rumo ao Topo! 2025, Nossa Escalada da Superação
> **Slogan Comum:** Juntos, no nosso papel, construímos o futuro no topo!
> 
> **1. Convite Digital Interativo:**
> 
> *   **Título:** PaperCare: Rumo ao Topo! 2025, Nossa Escalada da Superação
> *   **Texto:** Prepare-se para a Convenção de Vendas PaperCare 2025! Uma jornada de 4 dias rumo ao sucesso, com muito aprendizado, networking e momentos inesquecíveis. Juntos, vamos construir um futuro no topo!
> *   **Slogan:** Juntos, no nosso papel, construímos o futuro no topo!
> *   **CTA:** Confirme sua presença e prepare-se para decolar!
> *   **Hashtags:** #PaperCareNoTopo #Superação2025 #JuntosNoNossoPapel #ConvençãoDeVendas #RumoAoSucesso
> 
> **2. Teasers Semanais (E-mail, Mural, Intranet):**
> 
> *   **Título:** PaperCare: Rumo ao Topo! 2025, Nossa Escalada da Superação - Semana [Número da Semana]
> *   **Texto:**
>     *   **Semana 1:** Acredite, PaperCare! O topo te espera! Prepare-se para a Convenção de Vendas 2025, onde vamos traçar juntos o caminho para um ano de superação. Fique ligado para mais novidades!
>     *   **Semana 2:** Desafios nos impulsionam, PaperCare! Descubra como vamos escalar juntos os obstáculos e alcançar nossos objetivos na Convenção de Vendas 2025.
>     *   **Semana 3:** A união faz a força, PaperCare! Conecte-se com seus colegas, troque ideias e fortaleça o espírito de equipe na Convenção de Vendas 2025. O topo é nosso!
>     *   **Semana 4:** O futuro se constrói agora, PaperCare! Saiba tudo sobre as novidades e estratégias para 2025 na Convenção de Vendas. Imperdível!
> *   **Slogan:** Juntos, no nosso papel, construímos o futuro no topo!
> *   **Hashtags:** #PaperCareNoTopo #Superação2025 #JuntosNoNossoPapel #ConvençãoDeVendas #Motivação
> 
> **3. Vídeos Curtos com Depoimentos dos Líderes:**
> 
> *   **Título:** PaperCare: Rumo ao Topo! 2025, Nossa Escalada da Superação - Mensagem da Liderança
> *   **Roteiro:** (Ajustar para cada líder)
>     *   **Líder:** "Olá, time PaperCare! A Convenção de Vendas 2025 é o momento de celebrarmos nossas conquistas e nos prepararmos para um ano ainda mais desafiador. Acredito no potencial de cada um de vocês para superarmos qualquer obstáculo e levarmos a PaperCare ao topo!"
>     *   **Slogan:** "Juntos, no nosso papel, construímos o futuro no topo!"
> *   **Hashtags:** #PaperCareNoTopo #Superação2025 #Liderança #Motivação #ConvençãoDeVendas
> 
> **4. Banners e Cartazes (Com fotos dos colaboradores "escalando" desafios):**
> 
> *   **Título:** PaperCare: Rumo ao Topo! 2025, Nossa Escalada da Superação
> *   **Texto:** "Nosso time não tem medo de altura! 💪 Junte-se a nós na Convenção de Vendas PaperCare 2025 e prepare-se para escalar novos desafios e alcançar o sucesso!"
> *   **Slogan:** Juntos, no nosso papel, construímos o futuro no topo!
> *   **Hashtags:** #PaperCareNoTopo #Superação2025 #TimePaperCare #Desafios #ConvençãoDeVendas
> 
> **5. Cobertura do Evento em Tempo Real (Redes Sociais):**
> 
> *   **Posts:**
>     *   "Começou a Convenção de Vendas PaperCare 2025! Energia lá em cima para construirmos juntos um ano de superação! #PaperCareNoTopo #Superação2025"
>     *   "Dinâmica de grupo 'Escalada da Superação' rolando agora! Trabalho em equipe e muita garra para alcançar o topo! #JuntosNoNossoPapel #TimeUnido"
>     *   "Palestra incrível com [Nome do Palestrante]! Inspiração e motivação para encarar os desafios de 2025! #Motivação #Superação"
>     *   "Jantar de confraternização: celebrando as conquistas e fortalecendo os laços da equipe PaperCare! #Celebração #TimePaperCare"
> *   **Stories:**
>     *   Bastidores das atividades, entrevistas rápidas com os participantes, enquetes, etc.
> *   **Hashtags:** #PaperCareNoTopo #Superação2025 #JuntosNoNossoPapel #ConvençãoDeVendas #TimePaperCare
> 
> **6. E-mail de Agradecimento Pós-Evento:**
> 
> *   **Título:** PaperCare: Rumo ao Topo! Nossa Escalada da Superação - Convenção de Vendas 2025
> *   **Texto:** "Agradecemos a participação de todos na Convenção de Vendas PaperCare 2025! Foi um sucesso, e juntos traçamos um caminho inspirador rumo a um ano de superação e conquistas. Que a energia e o aprendizado desses dias nos impulsionem a alcançar o topo! Em breve, compartilharemos os melhores momentos e os próximos passos."
> *   **Slogan:** Juntos, no nosso papel, construímos o futuro no topo!
> *   **Hashtags:** #PaperCareNoTopo #Superação2025 #JuntosNoNossoPapel #ConvençãoDeVendas #Gratidão
> 


----------------------------------------------------------------

--- Resultado da Etapa de Revisão da Comunicação ---



> Aprovado! Podem seguir.


----------------------------------------------------------------
ASSISTENTE BUSINESS PARTY - IROTSUKA - 17/05/2025
#####################################################
#    LINKEDIN:https://www.linkedin.com/in/irotsuka/ #
#    GITHUB: https://github.com/i-otsuka            #
#####################################################
